In [1]:
# CIFAR-10 : 10개의 레이블, 6만장의 칼라 이미지(5만장 - train, 1만장 - test)
# airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck
# DENSE 레이어로만 분류작업1

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10



In [ ]:
NUM_CLASSES = 10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('train data')
print(x_train.shape)        #(50000, 32, 32, 3)
print(x_train.shape[0])
print(x_train.shape[3])     # 채널
print('test data')
print(x_test.shape)         # (10000, 32, 32, 3)

# print(x_train[0])
# print(y_train[0])     # frog

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.imshow(x_train[0], interpolation='bicubic')
plt.subplot(1, 3, 2)
plt.imshow(x_train[1], interpolation='bicubic')
plt.subplot(1, 3, 3)
plt.imshow(x_train[2], interpolation='bicubic')

x_train = x_train.astype('float32')  / 255.0
x_test = x_test.astype('float32')  / 255.0

y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

# print(x_train)
print(x_train[54,12,13,1])      # 0.36862746.  54번째 인덱스(이미지)의 12행 13열 중에 1번채널(초록색), 2번은 파란색 

In [ ]:
# 방법1 : Sequential API를 사용 (CNN)
model = Sequential([
                    Dense(512, input_shape=(32, 32, 3), activation='relu'),
                    Flatten(),
                    Dense(128, activation='relu'),
                    Dense(NUM_CLASSES, activation='softmax'),
])

print(model.summary())

In [ ]:
# 방법2 : Function API를 사용 (CNN x)
input_layer = Input((32, 32, 3))
x = Flatten()(input_layer)      # Input(shape=(32, 32, 3))
x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(input_layer, output_layer)
print(model.summary())

In [ ]:
# train
opt = Adam(lr = 0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=128, epochs=10, shuffle=True, verbose=2)
print('acc:%.4f' %(model.evaluate(x_test, y_test, batch_size=128)[1]))
print('loss:%.4f' %(model.evaluate(x_test, y_test, batch_size=128)[0]))

In [ ]:
CLASSES = np.array(["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"])

pred = model.predict(x_test[:10])
pred_single = CLASSES[np.argmax(pred, axis=-1)]
actual_single = CLASSES[np.argmax(y_test[:10], axis=-1)]
print('예측값 : ', pred_single)
print('실제값 : ', actual_single)
print('분류 실패 수 : ', (pred_single != actual_single).sum())

In [ ]:
# 시각화
fig = plt.figure(figsize= (15, 3))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

for i, idx in enumerate(range(len(x_test[:10]))):
  img = x_test[idx]
  ax = fig.add_subplot(1, len(x_test[:10]), i + 1)
  ax.axis('off')
  ax.text(0.5, -0.35, 'pred  =  ' + str(pred_single[idx]), fontsize=10,  ha = 'center', transform = ax.transAxes)
  ax.text(0.5, -0.7, 'act  =  ' + str(actual_single[idx]), fontsize=10,  ha = 'center', transform = ax.transAxes)
  ax.imshow(img)

plt.show()


In [ ]:
# CNN + Dense 레이어로만 분류작업
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, Activation, BatchNormalization, ReLU, LeakyReLU, MaxPool2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10

In [ ]:
NUM_CLASSES = 10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')  / 255.0
x_test = x_test.astype('float32')  / 255.0

y_train = to_categorical(y_train, NUM_CLASSES)
y_test = to_categorical(y_test, NUM_CLASSES)

In [ ]:
# function API, CNN + Dense
input_layer = Input(shape=(32, 32, 3))
conv_layer1 = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(input_layer)
conv_layer2 = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(conv_layer1)

flatten_layer = Flatten()(conv_layer2)

output_layer = Dense(units = 10, activation='softmax')(flatten_layer)
model = Model(input_layer, output_layer)
print(model.summary())

In [ ]:
# function API, CNN + Dense
input_layer = Input(shape=(32, 32, 3))
x = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(input_layer)
x = MaxPool2D(pool_size=(2, 2))(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters=64, kernel_size=3, strides=2, padding='same')(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Flatten()(x)

x = Dense(512)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x)

model = Model(input_layer, output_layer)

In [ ]:
# train
opt = Adam(lr = 0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=128, epochs=10, shuffle=True, verbose=2)
print('acc:%.4f' %(model.evaluate(x_test, y_test, batch_size=128)[1]))
print('loss:%.4f' %(model.evaluate(x_test, y_test, batch_size=128)[0]))